In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import sklearn
import sklearn.ensemble
import sklearn.linear_model
import sklearn.svm
import sklearn.naive_bayes
import xgboost
from sklearn.calibration import CalibratedClassifierCV

# from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

In [3]:
rf_clf = sklearn.ensemble.RandomForestClassifier(random_state=0, criterion='entropy', n_estimators=5)
xgb_clf = xgboost.XGBClassifier(random_state=0, n_estimators=50)
gdbt_clf = sklearn.ensemble.GradientBoostingClassifier(random_state=0, n_estimators=5)

lsvc_clf = CalibratedClassifierCV(sklearn.svm.LinearSVC(C=100, random_state=0), cv=5)
lcsvc_clf = sklearn.svm.LinearSVC(C=1000, random_state=0)
rsvc_clf = sklearn.svm.SVC(kernel='rbf', C=10, gamma='auto', random_state=0)
lr_clf = sklearn.linear_model.LogisticRegression(random_state=0, solver='lbfgs', multi_class='auto')
bnb_clf = sklearn.naive_bayes.BernoulliNB(alpha=100)

clf_base = {
    'randomforest': rf_clf,
    'SVM_linear': lsvc_clf,
    'logistic': lr_clf,
    'NB': bnb_clf,
    'xgboost': xgb_clf,
    "SVM_rbf": rsvc_clf,
}

clf_smote = {model + '+SMOTE': Pipeline([("smote", SMOTE()), ("clf", clf_base[model])]) for model in clf_base}

# params = {
#     'randomforest': {'n_estimators': [30, 50, 80], 'criterion': ['entropy', 'gini']},
#     'SVM': {},
#     'logistic': {'C': [1, 0.5],},
#     'NB': {'alpha': [100, 50, 10],},
#     'xgboost': {'n_estimators': [30, 50, 80],},
# }

clf_models = {**clf_base, **clf_smote}

### read active user data

In [4]:
df_user = pd.read_csv("./DSock/Active_Users.csv")
df_user["isPart"] = (df_user[["isMod", "isObserver", "isPuppet"]] == "t").any(axis=1)

print(f"{df_user.shape}")

(116, 57)


In [5]:
df_part = df_user[df_user["isPart"]]
df_covert = df_user[df_user["Strategy"] == "COVERT"]
df_overt = df_user[df_user["Strategy"] == "OVERT"]
df_unres = df_user[df_user["Strategy"] == "UNRESTRICTED"]

print(f"{df_part.shape} {df_covert.shape} {df_overt.shape} {df_unres.shape}")

(69, 57) (22, 57) (8, 57) (5, 57)


In [6]:
def get_data(df_part, df_sock):
    X = pd.concat([df_part, df_sock], axis=0)[["received_comments", "issued_posts"]].values
#     X = pd.concat([df_part, df_sock], axis=0)[["received_comments", "issued_posts", "issued_like", "issued_comment"]].values
    y = [0] * df_part.shape[0] + [1] * df_sock.shape[0]
    y = np.array(y)
    return X, y

In [7]:
import warnings
warnings.filterwarnings("ignore")

def get_metrics(X, y):
    res = {
        m: cross_validate(estimator=clf_models[m], X=X, y=y, cv=10, scoring=["f1", "precision", "recall", "accuracy",]) for m in clf_models
    }
    df_res = pd.DataFrame({
        m: pd.DataFrame(res[m]).mean()
        for m in clf_models
    })
    return df_res

In [8]:
res = {
    "COVERT": get_metrics(*get_data(df_part, df_covert)),
    "OVERT": get_metrics(*get_data(df_part, df_overt)),
    "UNRESTRICTED": get_metrics(*get_data(df_part, df_unres)),
    "ALLSOCKS": get_metrics(*get_data(df_part, pd.concat([df_covert, df_overt, df_unres], axis=0))),
}


In [9]:
df_res = pd.concat(res, keys=res.keys(), axis=0)
display(df_res)
df_res.to_csv("./res/par_socks_res.csv")

randomforest  SVM_linear  logistic        NB  \
COVERT       fit_time            0.006351    0.021740  0.002569  0.000502   
             score_time          0.002384    0.002037  0.001406  0.001446   
             test_f1             0.033333    0.233333  0.066667  0.000000   
             test_precision      0.025000    0.350000  0.100000  0.000000   
             test_recall         0.050000    0.183333  0.050000  0.000000   
             test_accuracy       0.592222    0.781111  0.703333  0.758889   
OVERT        fit_time            0.006737    0.015795  0.002785  0.000495   
             score_time          0.002559    0.002066  0.001472  0.001466   
             test_f1             0.000000    0.000000  0.000000  0.000000   
             test_precision      0.000000    0.000000  0.000000  0.000000   
             test_recall         0.000000    0.000000  0.000000  0.000000   
             test_accuracy       0.803571    0.898214  0.873214  0.898214   
UNRESTRICTED fit_time            0.006768    0.008305  0.003178  0.000487   
             score_time          0.002583    0.001039  0.001415  0.001459   
             test_f1             0.100000    0.000000  0.000000  0.000000   
             test_precision      0.100000    0.000000  0.000000  0.000000   
             test_recall         0.100000    0.000000  0.000000  0.000000   
             test_accuracy       0.867857    1.000000  0.935714  0.935714   
ALLSOCKS     fit_time            0.006461    0.016409  0.002211  0.000488   
             score_time          0.002328    0.001913  0.001399  0.001433   
             test_f1             0.163268    0.120000  0.178333  0.000000   
             test_precision      0.186905    0.150000  0.225000  0.000000   
             test_recall         0.158333    0.100000  0.150000  0.000000   
             test_accuracy       0.460000    0.645455  0.636364  0.664545   

                              xgboost   SVM_rbf  randomforest+SMOTE  \
COVERT       fit_time        0.012508  0.000838            0.007936   
             score_time      0.002432  0.002011            0.002570   
             test_f1         0.033333  0.000000            0.187619   
             test_precision  0.025000  0.000000            0.165000   
             test_recall     0.050000  0.000000            0.250000   
             test_accuracy   0.536667  0.570000            0.558889   
OVERT        fit_time        0.005739  0.000735            0.007670   
             score_time      0.002395  0.001988            0.002563   
             test_f1         0.000000  0.000000            0.166667   
             test_precision  0.000000  0.000000            0.116667   
             test_recall     0.000000  0.000000            0.300000   
             test_accuracy   0.830357  0.803571            0.716071   
UNRESTRICTED fit_time        0.004915  0.000700            0.000516   
             score_time      0.002410  0.001992            0.000000   
             test_f1         0.000000  0.000000                 NaN   
             test_precision  0.000000  0.000000                 NaN   
             test_recall     0.000000  0.000000                 NaN   
             test_accuracy   0.908929  0.866071                 NaN   
ALLSOCKS     fit_time        0.009101  0.000915            0.007930   
             score_time      0.002861  0.002001            0.002557   
             test_f1         0.179935  0.053333            0.280238   
             test_precision  0.186905  0.036364            0.256364   
             test_recall     0.183333  0.100000            0.350000   
             test_accuracy   0.460000  0.460000            0.480000   

                             SVM_linear+SMOTE  logistic+SMOTE  NB+SMOTE  \
COVERT       fit_time                0.020084        0.003491  0.001261   
             score_time              0.002024        0.001402  0.001520   
             test_f1                 0.552143        0.395000  0.484286   
             test_precision